In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
# import sklearn

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# import tensorflow as tf
# from tensorflow import keras

# import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [2]:
conn = sqlite3.connect('NBA-Boxscore-Database.sqlite')

In [3]:
query = 'SELECT * from game_info'
game_info = pd.read_sql(query, con=conn)
game_info

,game_id,season,date,away_team,away_score,home_team,home_score,result
0,131410290001,1314,2013-10-29,ORL,87,IND,97,1
1,131410290002,1314,2013-10-29,CHI,95,MIA,107,1
2,131410290003,1314,2013-10-29,LAC,103,LAL,116,1
3,131410300004,1314,2013-10-30,BRK,94,CLE,98,1
4,131410300005,1314,2013-10-30,BOS,87,TOR,93,1
...,...,...,...,...,...,...,...,...
11974,222304091226,2223,2023-04-09,UTA,117,LAL,128,1
11975,222304091227,2223,2023-04-09,NOP,108,MIN,113,1
11976,222304091228,2223,2023-04-09,MEM,100,OKC,115,1
11977,222304091229,2223,2023-04-09,LAC,119,PHO,114,0


In [4]:
query = 'SELECT * from team_stats'
team_stats = pd.read_sql(query, con=conn)
team_stats

,game_id,team,MP,FG,FGA,FGp,3P,3PA,3Pp,FT,FTA,FTp,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PM,TSp,eFGp,3PAr,FTr,ORBp,DRBp,TRBp,ASTp,STLp,BLKp,TOVp,USGp,ORtg,DRtg,BPM
0,131410290001,ORL,240,36.0,93.0,0.387,9.0,19.0,0.474,6.0,10.0,0.600,13.0,26.0,39.0,17.0,10.0,6.0,17.0,26.0,87.0,None,0.447,0.435,0.204,0.108,27.7,72.2,47.0,47.2,10.5,11.1,14.9,100.0,91.6,102.1,None
1,131410290001,IND,240,34.0,71.0,0.479,7.0,17.0,0.412,22.0,32.0,0.688,10.0,34.0,44.0,17.0,4.0,18.0,20.0,13.0,97.0,None,0.570,0.528,0.239,0.451,27.8,72.3,53.0,50.0,4.2,24.3,19.0,100.0,102.1,91.6,None
2,131410290002,CHI,240,35.0,83.0,0.422,7.0,26.0,0.269,18.0,23.0,0.783,11.0,30.0,41.0,23.0,11.0,4.0,18.0,27.0,95.0,None,0.510,0.464,0.313,0.277,23.9,85.7,50.6,65.7,11.3,7.7,16.2,100.0,97.9,110.2,None
3,131410290002,MIA,240,37.0,72.0,0.514,11.0,20.0,0.550,22.0,29.0,0.759,5.0,35.0,40.0,26.0,10.0,7.0,18.0,21.0,107.0,None,0.631,0.590,0.278,0.403,14.3,76.1,49.4,70.3,10.3,12.3,17.5,100.0,110.2,97.9,None
4,131410290003,LAC,240,41.0,83.0,0.494,8.0,21.0,0.381,13.0,23.0,0.565,10.0,30.0,40.0,27.0,11.0,4.0,16.0,21.0,103.0,None,0.553,0.542,0.253,0.277,22.7,62.5,43.5,65.9,11.0,6.3,14.7,100.0,102.6,115.6,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23953,222304091228,OKC,240,45.0,90.0,0.500,12.0,36.0,0.333,13.0,16.0,0.813,8.0,41.0,49.0,30.0,5.0,2.0,14.0,17.0,115.0,None,0.593,0.567,0.400,0.178,20.0,78.8,53.3,66.7,5.0,3.6,12.6,100.0,114.9,99.9,None
23954,222304091229,LAC,240,49.0,100.0,0.490,5.0,23.0,0.217,16.0,24.0,0.667,14.0,39.0,53.0,22.0,3.0,5.0,10.0,18.0,119.0,None,0.538,0.515,0.230,0.240,28.6,76.5,53.0,44.9,2.9,8.1,8.3,100.0,116.4,111.5,None
23955,222304091229,PHO,240,42.0,99.0,0.424,14.0,37.0,0.378,16.0,22.0,0.727,12.0,35.0,47.0,29.0,4.0,3.0,7.0,21.0,114.0,None,0.524,0.495,0.374,0.222,23.5,71.4,47.0,69.0,3.9,3.9,6.1,100.0,111.5,116.4,None
23956,222304091230,GSW,240,58.0,96.0,0.604,27.0,49.0,0.551,14.0,16.0,0.875,9.0,49.0,58.0,47.0,13.0,6.0,15.0,18.0,157.0,None,0.762,0.745,0.510,0.167,27.3,89.1,65.9,81.0,11.9,10.0,12.7,100.0,143.9,92.6,None


In [5]:
game_ids = list(team_stats['game_id'].unique())

In [6]:
stats_df = team_stats.drop(['MP', 'PM', 'USGp', 'ORtg', 'DRtg', 'BPM'], axis=1)
stats_df

,game_id,team,FG,FGA,FGp,3P,3PA,3Pp,FT,FTA,FTp,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,TSp,eFGp,3PAr,FTr,ORBp,DRBp,TRBp,ASTp,STLp,BLKp,TOVp
0,131410290001,ORL,36.0,93.0,0.387,9.0,19.0,0.474,6.0,10.0,0.600,13.0,26.0,39.0,17.0,10.0,6.0,17.0,26.0,87.0,0.447,0.435,0.204,0.108,27.7,72.2,47.0,47.2,10.5,11.1,14.9
1,131410290001,IND,34.0,71.0,0.479,7.0,17.0,0.412,22.0,32.0,0.688,10.0,34.0,44.0,17.0,4.0,18.0,20.0,13.0,97.0,0.570,0.528,0.239,0.451,27.8,72.3,53.0,50.0,4.2,24.3,19.0
2,131410290002,CHI,35.0,83.0,0.422,7.0,26.0,0.269,18.0,23.0,0.783,11.0,30.0,41.0,23.0,11.0,4.0,18.0,27.0,95.0,0.510,0.464,0.313,0.277,23.9,85.7,50.6,65.7,11.3,7.7,16.2
3,131410290002,MIA,37.0,72.0,0.514,11.0,20.0,0.550,22.0,29.0,0.759,5.0,35.0,40.0,26.0,10.0,7.0,18.0,21.0,107.0,0.631,0.590,0.278,0.403,14.3,76.1,49.4,70.3,10.3,12.3,17.5
4,131410290003,LAC,41.0,83.0,0.494,8.0,21.0,0.381,13.0,23.0,0.565,10.0,30.0,40.0,27.0,11.0,4.0,16.0,21.0,103.0,0.553,0.542,0.253,0.277,22.7,62.5,43.5,65.9,11.0,6.3,14.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23953,222304091228,OKC,45.0,90.0,0.500,12.0,36.0,0.333,13.0,16.0,0.813,8.0,41.0,49.0,30.0,5.0,2.0,14.0,17.0,115.0,0.593,0.567,0.400,0.178,20.0,78.8,53.3,66.7,5.0,3.6,12.6
23954,222304091229,LAC,49.0,100.0,0.490,5.0,23.0,0.217,16.0,24.0,0.667,14.0,39.0,53.0,22.0,3.0,5.0,10.0,18.0,119.0,0.538,0.515,0.230,0.240,28.6,76.5,53.0,44.9,2.9,8.1,8.3
23955,222304091229,PHO,42.0,99.0,0.424,14.0,37.0,0.378,16.0,22.0,0.727,12.0,35.0,47.0,29.0,4.0,3.0,7.0,21.0,114.0,0.524,0.495,0.374,0.222,23.5,71.4,47.0,69.0,3.9,3.9,6.1
23956,222304091230,GSW,58.0,96.0,0.604,27.0,49.0,0.551,14.0,16.0,0.875,9.0,49.0,58.0,47.0,13.0,6.0,15.0,18.0,157.0,0.762,0.745,0.510,0.167,27.3,89.1,65.9,81.0,11.9,10.0,12.7


In [7]:
def create_gid_query(game_id, date, lim, team='away'):
    away_gid_query = f'''
    SELECT gi.game_id
    FROM game_info gi
    WHERE (gi.away_team = (SELECT away_team FROM game_info WHERE game_id = '{game_id}')
            AND gi.date < '{date}')
       OR (gi.home_team = (SELECT away_team FROM game_info WHERE game_id = '{game_id}')
            AND gi.date < '{date}')
    ORDER BY gi.date DESC
    LIMIT {lim};
    '''
    
    home_gid_query = f'''
    SELECT gi.game_id
    FROM game_info gi
    WHERE (gi.away_team = (SELECT home_team FROM game_info WHERE game_id = '{game_id}')
            AND gi.date < '{date}')
       OR (gi.home_team = (SELECT home_team FROM game_info WHERE game_id = '{game_id}')
            AND gi.date < '{date}')
    ORDER BY gi.date DESC
    LIMIT {lim};
    '''
    
    if team == 'away':
        return away_gid_query
    elif team == 'home':
        return home_gid_query

In [8]:
def create_stats_df(query, conn, stats, team='away'):
    
    gid_query_df = pd.read_sql(query, con=conn)
    stat_df = gid_query_df.merge(stats)
    
    if team == 'away':
        stat_df = stat_df[stat_df['team'] == away_team].reset_index(drop=True)
        stat_agg = stat_df.loc[:,'FG':'TOVp'].mean()
        stat_agg = stat_agg.to_frame().T
        columns = [f'a_{col}' for col in stat_agg.columns]
        col_name_dict = dict(zip(stat_agg.columns, columns))
        stat_agg.rename(columns=col_name_dict, inplace=True)
        
    elif team =='home':
        stat_df = stat_df[stat_df['team'] == home_team].reset_index(drop=True)
        stat_agg = stat_df.loc[:,'FG':'TOVp'].mean()
        stat_agg = stat_agg.to_frame().T
        columns = [f'h_{col}' for col in stat_agg.columns]
        col_name_dict = dict(zip(stat_agg.columns, columns))
        stat_agg.rename(columns=col_name_dict, inplace=True)
    
    return stat_agg

In [9]:
stats_df = team_stats.drop(['MP', 'PM', 'USGp', 'ORtg', 'DRtg', 'BPM'], axis=1)
final_df_columns = list(game_info.columns) + [f'a_{column}' for column in stats_df.loc[:,'FG':'TOVp']] + [f'h_{column}' for column in stats_df.loc[:,'FG':'TOVp']]
team_stats_full_10 = pd.DataFrame(columns=final_df_columns)

for game_id in game_ids:

    date = game_info[game_info['game_id'] == game_id]['date'].values[0]
    away_team = game_info[game_info['game_id'] == game_id]['away_team'].values[0]
    home_team = game_info[game_info['game_id'] == game_id]['home_team'].values[0]

    away_gid_query = create_gid_query(game_id = game_id, date=date, lim=10, team='away')
    away_stats = create_stats_df(query=away_gid_query, conn=conn, stats=stats_df, team='away')

    home_gid_query = create_gid_query(game_id=game_id, date=date, lim=10, team='home')
    home_stats = create_stats_df(query=home_gid_query, conn=conn, stats=stats_df, team='home')
    
    agg_stats = pd.concat([away_stats, home_stats], axis=1)
    gid_info = game_info[game_info['game_id'] == game_id].reset_index(drop=True)
    
    stats = pd.concat([gid_info, agg_stats], axis=1)
    team_stats_full_10 = team_stats_full_10.append(stats, ignore_index=True)

team_stats_full_10.to_csv('team_stats_full_10.csv')

In [11]:
stats_df = team_stats.drop(['MP', 'PM', 'USGp', 'ORtg', 'DRtg', 'BPM'], axis=1)
final_df_columns = list(game_info.columns) + [f'a_{column}' for column in stats_df.loc[:,'FG':'TOVp']] + [f'h_{column}' for column in stats_df.loc[:,'FG':'TOVp']]
team_stats_full_20 = pd.DataFrame(columns=final_df_columns)

for game_id in game_ids:

    date = game_info[game_info['game_id'] == game_id]['date'].values[0]
    away_team = game_info[game_info['game_id'] == game_id]['away_team'].values[0]
    home_team = game_info[game_info['game_id'] == game_id]['home_team'].values[0]

    away_gid_query = create_gid_query(game_id = game_id, date=date, lim=20, team='away')
    away_stats = create_stats_df(query=away_gid_query, conn=conn, stats=stats_df, team='away')

    home_gid_query = create_gid_query(game_id=game_id, date=date, lim=20, team='home')
    home_stats = create_stats_df(query=home_gid_query, conn=conn, stats=stats_df, team='home')
    
    agg_stats = pd.concat([away_stats, home_stats], axis=1)
    gid_info = game_info[game_info['game_id'] == game_id].reset_index(drop=True)
    
    stats = pd.concat([gid_info, agg_stats], axis=1)
    team_stats_full_20 = team_stats_full_20.append(stats, ignore_index=True)

team_stats_full_20.to_csv('team_stats_full_20.csv')

In [12]:
stats_df = team_stats.drop(['MP', 'PM', 'USGp', 'ORtg', 'DRtg', 'BPM'], axis=1)
final_df_columns = list(game_info.columns) + [f'a_{column}' for column in stats_df.loc[:,'FG':'TOVp']] + [f'h_{column}' for column in stats_df.loc[:,'FG':'TOVp']]
team_stats_full_30 = pd.DataFrame(columns=final_df_columns)

for game_id in game_ids:

    date = game_info[game_info['game_id'] == game_id]['date'].values[0]
    away_team = game_info[game_info['game_id'] == game_id]['away_team'].values[0]
    home_team = game_info[game_info['game_id'] == game_id]['home_team'].values[0]

    away_gid_query = create_gid_query(game_id = game_id, date=date, lim=30, team='away')
    away_stats = create_stats_df(query=away_gid_query, conn=conn, stats=stats_df, team='away')

    home_gid_query = create_gid_query(game_id=game_id, date=date, lim=30, team='home')
    home_stats = create_stats_df(query=home_gid_query, conn=conn, stats=stats_df, team='home')
    
    agg_stats = pd.concat([away_stats, home_stats], axis=1)
    gid_info = game_info[game_info['game_id'] == game_id].reset_index(drop=True)
    
    stats = pd.concat([gid_info, agg_stats], axis=1)
    team_stats_full_30 = team_stats_full_30.append(stats, ignore_index=True)

team_stats_full_30.to_csv('team_stats_full_30.csv')